In [ ]:
import ollama
import pandas as pd
from tqdm.notebook import tqdm
from rich import print as p

In [ ]:
COLS_TO_KEEP = ['text', 'labels_int']

ID_TO_CAT = {
    0.: 'negative',
    1.: 'neutral',
    2.: 'positive'
}

PROMPT_TEMPLATE = """
Você é um modelo de linguagem treinado para analisar sentenças e classificá-las 
como positivas, neutras ou negativas. Vou fornecer uma sentença, e você precisa 
retornar apenas uma das seguintes categorias: positive, neutral ou negative.

Sentença: {text}

Resposta com apenas uma palavra:
"""

In [ ]:
def id_to_categorical(data, id_col):
    data['labels'] = data[id_col].map(ID_TO_CAT)
    return data

def get_opinion(model, text):
    response = ollama.chat(
        model=model,
        messages=[{
            'role': 'user',
            'content': PROMPT_TEMPLATE.format(text=text)
        }]
    )
    return response['message']['content'].lower().strip()

def normalize_opinion(opinion):
    return (
        'positive' if 'positive' in opinion 
        else 'negative' if 'negative' in opinion 
        else 'neutral'
    )

In [ ]:
for data in ('facebook', 'twitter'):
    df = pd.read_csv(f'../data/processed/{data}_cachaca.csv')
    df = id_to_categorical(
        df[df.labels_int.notnull()][COLS_TO_KEEP], 'labels_int'
    )
    for model in ('llama3', 'mistral'):
        opinions = []
        for text in tqdm(df.text):
            opinions.append(normalize_opinion(get_opinion(model, text)))
        df[model] = opinions
    df.to_csv(f'../data/opinions/{data}_cachaca.csv', index=False)